In [1]:
import init
from uuoskit import uuosapi
from uuoskit import wasmcompiler
init.init()
test_account1 = init.test_account1

In [2]:
uuosapi.get_account(test_account1)

{'account_name': 'helloworld11',
 'head_block_num': 8058283,
 'head_block_time': '2021-11-14T13:16:03.000',
 'privileged': False,
 'last_code_update': '2021-11-14T13:15:00.000',
 'created': '2021-02-07T17:59:33.000',
 'core_liquid_balance': '100000840.5901 EOS',
 'ram_quota': 15730017,
 'net_weight': '83716500642',
 'cpu_weight': '334875932576',
 'net_limit': {'used': 30009,
  'available': '27480957603792',
  'max': '27480957633801',
  'last_usage_update_time': {'slot': 230070300},
  'current_used': 29987},
 'cpu_limit': {'used': 16035,
  'available': '39306674411696',
  'max': '39306674427731',
  'last_usage_update_time': {'slot': 230070300},
  'current_used': 16023},
 'ram_usage': 209399,
 'permissions': [{'perm_name': 'active',
   'parent': 'owner',
   'required_auth': {'threshold': 1,
    'keys': [{'key': 'EOS6AjF6hvF7GSuSd4sCgfPKq5uWaXvGM2aQtEUCwmEHygQaqxBSV',
      'weight': 1}],
    'accounts': [{'permission': {'actor': 'helloworld11',
       'permission': 'eosio.code'},
      '

In [3]:
a = {
    "account": test_account1,
    "permission": "active",
    "parent": "owner",
    "auth": {
        "threshold": 1,
        "keys": [
            {
                "key": "EOS6AjF6hvF7GSuSd4sCgfPKq5uWaXvGM2aQtEUCwmEHygQaqxBSV",
                "weight": 1
            },
        ],
        "accounts": [{"permission":{"actor":test_account1,"permission":"eosio.code"},"weight":1}],
        "waits": []
    }
}
r = uuosapi.push_action('eosio', 'updateauth', a, {test_account1:'active'})
print('done!')

done!


In [4]:
test_account1

'helloworld11'

In [5]:
code = r'''
package main

import "github.com/uuosio/chain"

func main() {
    receiver, _, action := chain.GetApplyArgs()
    if action == chain.NewName("sayhello") {
        a := chain.NewAction(chain.PermissionLevel{receiver, chain.N("active")}, chain.N("eosio.token"), chain.N("transfer"))
        a.AddPermission(chain.N("helloworld11"), chain.ActiveName)

        t := chain.Transfer{}
        t.From = chain.N("helloworld11")
        t.To = chain.N("eosio")
        
        // Send 1.0 EOS
        t.Quantity.Amount = 10000;
        t.Quantity.Symbol = chain.NewSymbol("EOS", 4);

        a.Data = t.Pack()
        a.Send()
    }
    
}
'''
code, abi = uuosapi.compile(test_account1, code, src_type=2)
uuosapi.deploy_code(test_account1, code, vm_type=0)
print('done!')

done!


In [6]:
uuosapi.get_balance(test_account1), uuosapi.get_balance('eosio')

(100000840.5901, 589987472.6273)

In [7]:
r = uuosapi.push_action(test_account1, 'sayhello', b'hello,world', {test_account1:'active'})
print(r['processed']['action_traces'][0]['console'])
print(r['processed']['elapsed'])


2962


In [8]:
uuosapi.get_balance(test_account1), uuosapi.get_balance('eosio')

(100000839.5901, 589987473.6273)